In [3]:
import sklearn as sk
from sklearn.linear_model import LogisticRegression
import pandas as pd 
import numpy as np 
import io
import requests
import csv

In [326]:
url_train_logits = 'https://raw.githubusercontent.com/csu-signal/loan-word-detection/main/Datasets/production_train_test/real_distribution/train_withlogits_realdist.csv'
#url_train_logits = 'https://raw.githubusercontent.com/csu-signal/loan-word-detection/main/Datasets/production_train_test/alldata/train_withlogits_alldata.csv'
url_test_logits = 'https://raw.githubusercontent.com/csu-signal/loan-word-detection/main/Datasets/production_train_test/real_distribution/test_withlogits_realdist.csv'

#url_test_logits = 'https://raw.githubusercontent.com/csu-signal/loan-word-detection/main/Datasets/production_train_test/alldata/test_withlogits_alldata.csv'


url_train_logits = requests.get(url_train_logits).content
url_test_logits = requests.get(url_test_logits).content
train=pd.read_csv(io.StringIO(url_train_logits.decode('utf-8')))
test = pd.read_csv(io.StringIO(url_test_logits.decode('utf-8')))


In [327]:
train.shape, test.shape

((4200, 23), (462, 23))

In [328]:
train 

,Unnamed: 0,Unnamed: 0.1,loan_word,original_word,loan_word_epitran,original_word_epitran,loan_english,original_english,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,...,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,label,label_bin,mbert_cos_similarity,xlm_cos_similarity,features_loan,features_orig,DNN_logits,CNN_logits
0,0,2592,निराशा,افسردگی,niraːʃaː,ɒfsrdɡj,Disappointment,Depression,1.000000,0.857143,...,0.342262,7,synonym,0,0.333955,0.605581,"[-1, 1, 1, -1, -1, -1, 1, 0, 1, -1, -1, 1, 1, ...","[1, 1, -1, 1, -1, -1, -1, 0, 1, -1, -1, 0, -1,...",-14.651514,-50.400696
1,1,218,कोफ़्ता,کوفته,koftaː,kvfth,kofta,Meatballs,0.500000,0.400000,...,0.112500,7,loan,1,0.623054,0.525325,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, -1,...","[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, -1,...",6.762000,4.931340
2,2,2454,फ़िरंगी,بحران‌زا,firəŋɡiː,bhrɒn‌zɒ,firangi,Crisis,0.875000,0.285714,...,0.151786,8,hard_negative,0,0.534576,0.702203,"[-1, -1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -...","[-1, -1, 1, -1, -1, -1, -1, 0, 1, -1, -1, 1, -...",-16.894947,-37.755050
3,3,2655,अभी,ماتم زده,ab̤iː,mɒtm zdh,Now,Mourning,1.000000,0.857143,...,0.636905,8,synonym,0,0.512158,0.545001,"[1, 1, -1, 1, -1, -1, -1, 0, 1, -1, -1, 0, -1,...","[-1, 1, 1, -1, -1, -1, 1, 0, 1, -1, -1, 1, -1,...",-24.559683,-60.811077
4,4,926,मर्दुम,تپ دیق,mərdum,tp djɣ,Mardum,تپ دیق,0.833333,0.833333,...,0.340278,6,random,0,0.395930,0.702725,"[-1, 1, 1, -1, -1, -1, 1, 0, 1, -1, -1, 1, -1,...","[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ...",-9.449596,-4.686728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,4195,3791,अप्रिय,ناخوشایند,aprij,nɒxvʃɒjnd,unpleasant,Unpleasant,0.888889,0.555556,...,0.511574,9,synonym,0,0.450621,0.619644,"[1, 1, -1, 1, -1, -1, -1, 0, 1, -1, -1, 0, -1,...","[-1, 1, 1, -1, -1, -1, 1, 0, 1, -1, -1, 1, 1, ...",-30.052359,-61.183994
4196,4196,1250,साहिल,افسوس,saːɦil,ɒfsvs,Sahil,Alas,1.000000,1.000000,...,0.266667,5,random,0,0.464280,0.785964,"[-1, -1, 1, 1, -1, -1, -1, 0, -1, -1, -1, 1, 1...","[1, 1, -1, 1, -1, -1, -1, 0, 1, -1, -1, 0, -1,...",-19.979090,-2.929000
4197,4197,532,तहसील,طبیب,təɦsiːl,tbjb,Tehsil,Medical,0.857143,0.666667,...,0.420139,5,random,0,0.329295,0.405548,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ...","[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ...",-9.914829,-36.352913
4198,4198,756,फ़क़त,کباب,fəqət,kbɒb,right,Kebab,1.000000,0.800000,...,0.333333,5,random,0,0.370624,0.717403,"[-1, -1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -...","[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, -1,...",-10.867018,-4.327223


In [329]:

# train['DNN_logits'] = train['DNN_logits'] / train['DNN_logits'].max()
# train['CNN_logits'] = train['CNN_logits'] / train['DNN_logits'].max()
train['DNN_logits'] = (train['DNN_logits']-train['DNN_logits'].min()) / (train['DNN_logits'].max()-train['DNN_logits'].min())
train['CNN_logits'] = (train['CNN_logits']-train['CNN_logits'].min()) / (train['CNN_logits'].max()-train['CNN_logits'].min())

test['DNN_logits'] = (test['DNN_logits']-test['DNN_logits'].min()) / (test['DNN_logits'].max()-test['DNN_logits'].min())
test['CNN_logits'] = (test['CNN_logits']-test['CNN_logits'].min()) / (test['CNN_logits'].max()-test['CNN_logits'].min())

In [330]:
test

,Unnamed: 0,Unnamed: 0.1,loan_word,original_word,loan_word_epitran,original_word_epitran,loan_english,original_english,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,...,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,label,label_bin,mbert_cos_similarity,xlm_cos_similarity,features_loan,features_orig,DNN_logits,CNN_logits
0,0,421,ज़रूरत,ضرورت,zəruːrət,zrvrt,need,Necessity,0.500000,0.428571,...,0.336310,6,loan,1,0.558685,0.735102,"[-1, -1, 1, 1, -1, -1, -1, 0, 1, -1, -1, 1, 1,...","[-1, -1, 1, 1, -1, -1, -1, 0, 1, -1, -1, 1, 1,...",0.912376,0.856959
1,1,63,आफ़त,آفت,aːfət,ɒft,disaster,آفت,0.600000,0.250000,...,0.281250,4,loan,1,0.478023,0.561306,"[1, 1, -1, 1, -1, -1, -1, 0, 1, -1, -1, 0, -1,...","[1, 1, -1, 1, -1, -1, -1, 0, 1, -1, -1, 0, -1,...",0.912065,0.804745
2,2,3499,लाल,زال,laːl,zɒl,Red,still,0.750000,0.333333,...,0.069444,3,hard_negative,0,0.591443,0.788639,"[-1, 1, 1, 1, -1, 1, -1, 0, 1, -1, -1, 1, 1, -...","[-1, -1, 1, 1, -1, -1, -1, 0, 1, -1, -1, 1, 1,...",0.629748,0.417709
3,3,1119,वर्ज़िश,تلخی,vərziʃ,tlxj,work,Bitter,1.000000,0.833333,...,0.440972,7,random,0,0.361615,0.738731,"[-1, -1, 1, 1, -1, -1, -1, 1, 1, -1, -1, 1, -1...","[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ...",0.608540,0.472363
4,4,4413,परेशान,ناراحت,pəreʃaːn,nɒrɒht,Worried,I'm upset,0.875000,0.571429,...,0.244048,6,synonym,0,0.599549,0.757741,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ...","[-1, 1, 1, -1, -1, -1, 1, 0, 1, -1, -1, 1, 1, ...",0.611787,0.467649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,457,561,भयंकर,ترسناک,b̤əjəŋkər,trsnɒk,Terrifying,Scary,0.888889,0.625000,...,0.419271,6,synonym,0,0.336027,0.865532,"[-1, -1, 1, -1, -1, -1, -1, 0, 1, 1, -1, 1, -1...","[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ...",0.507481,0.237777
458,458,329,गुंजाइश,رفو,ɡuɲd͡ʒaːiʃ,rfv,request,Refo,1.000000,1.000000,...,0.681548,7,random,0,0.456615,0.767777,"[-1, -1, 1, -1, -1, -1, -1, 0, 1, -1, -1, -1, ...","[-1, 1, 1, 1, 0, -1, -1, 0, 1, -1, -1, 1, 1, -...",0.580000,0.483072
459,459,516,तराशा,تراشه,təraːʃaː,trɒʃh,sculpt,Chip,0.625000,0.333333,...,0.229167,5,loan,1,0.503085,0.791900,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ...","[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ...",0.908457,0.830639
460,460,1010,असली,اصل,asliː,ɒsl,Real,Original,0.600000,0.250000,...,0.270833,4,synonym,0,0.484934,0.515540,"[1, 1, -1, 1, -1, -1, -1, 0, 1, -1, -1, 0, -1,...","[1, 1, -1, 1, -1, -1, -1, 0, 1, -1, -1, 0, -1,...",0.613030,0.481042


In [331]:
# features =  ['DNN_logits', 'CNN_logits','Fast Levenshtein Distance Div Maxlen',
#        'Dolgo Prime Distance Div Maxlen', 'Feature Edit Distance Div Maxlen',
#        'Hamming Feature Distance Div Maxlen',
#        'Weighted Feature Distance Div Maxlen',
#        'Partial Hamming Feature Distance Div Maxlen', 'plain Levenshtein',
#           'mbert_cos_similarity', 'xlm_cos_similarity',
          
#         ]

features =  ['Fast Levenshtein Distance Div Maxlen',
       'Dolgo Prime Distance Div Maxlen', 'Feature Edit Distance Div Maxlen',
       'Hamming Feature Distance Div Maxlen',
       'Weighted Feature Distance Div Maxlen',
       'Partial Hamming Feature Distance Div Maxlen', 'plain Levenshtein' ,'mbert_cos_similarity', 'xlm_cos_similarity',
           
        ]
# features =  ['Fast Levenshtein', 'Dolgo Prime Distance',
#        'Feature Edit Distance', 'Hamming Feature Distance',
#        'Weighted Feature Distance', 'Fast Levenshtein Distance Div Maxlen', 
#         ]

labels = ['label_bin']
logits = ['DNN_logits', 'CNN_logits']

In [332]:
# standardize the logits separately because other features have already been standardized

# def normalize(df):
#     result = df.copy()
#     for feature_name in df.columns:
#         max_value = df[feature_name].max()
#         min_value = df[feature_name].min()
#         result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
#     return result


In [333]:
x_train = train[features].values
y_train = train[labels].values.ravel()
x_test = test[features].values
y_test = test[labels].values.ravel()

In [334]:
x_train, y_train
x_test

array([[0.5       , 0.42857143, 0.30654762, ..., 6.        , 0.55868465,
        0.73510247],
       [0.6       , 0.25      , 0.25520833, ..., 4.        , 0.47802341,
        0.56130552],
       [0.75      , 0.33333333, 0.06944444, ..., 3.        , 0.59144282,
        0.78863871],
       ...,
       [0.625     , 0.33333333, 0.21180556, ..., 5.        , 0.50308508,
        0.79190004],
       [0.6       , 0.25      , 0.24479167, ..., 4.        , 0.48493394,
        0.51554048],
       [1.        , 0.8       , 0.375     , ..., 6.        , 0.40697962,
        0.45941818]])

In [335]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((4200, 9), (4200,), (462, 9), (462,))

In [336]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


In [337]:
#try after standardizing the data including the logits. 
# stand= StandardScaler()
# fit = stand.fit(x_train)
# x_train = fit.transform(x_train)

# x_test = fit.transform(x_test)

In [338]:
LR = LogisticRegression(random_state=1, solver='lbfgs', multi_class='ovr', max_iter=500 ).fit(x_train, y_train)

In [339]:
y_pred = LR.predict(x_test)

In [340]:
print("f1-score : ", f1_score(y_test, y_pred ))
print("precision : ",precision_score(y_test, y_pred))
print("recall : ",recall_score(y_test, y_pred )) 
print("accuracy : ",accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

f1-score :  0.8904109589041096
precision :  0.87248322147651
recall :  0.9090909090909091
accuracy :  0.9307359307359307
              precision    recall  f1-score   support

           0       0.96      0.94      0.95       319
           1       0.87      0.91      0.89       143

    accuracy                           0.93       462
   macro avg       0.92      0.92      0.92       462
weighted avg       0.93      0.93      0.93       462

[[300  19]
 [ 13 130]]


In [341]:
precision, recall, fscore, support = score(y_pred, y_test, pos_label=1)

In [342]:
print('fscore: {}'.format(fscore))
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print("accuracy : ",accuracy_score(y_test, y_pred))

print('support: {}'.format(support))

fscore: [0.94936709 0.89041096]
precision: [0.94043887 0.90909091]
recall: [0.95846645 0.87248322]
accuracy :  0.9307359307359307
support: [313 149]


In [343]:
unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])

In [344]:
tp = np.array(np.where(unq == 3)).tolist()[0]
fp = np.array(np.where(unq == 1)).tolist()[0]
tn = np.array(np.where(unq == 0)).tolist()[0]
fn = np.array(np.where(unq == 2)).tolist()[0]

In [345]:
test.iloc[fp,:]

,Unnamed: 0,Unnamed: 0.1,loan_word,original_word,loan_word_epitran,original_word_epitran,loan_english,original_english,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,...,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,label,label_bin,mbert_cos_similarity,xlm_cos_similarity,features_loan,features_orig,DNN_logits,CNN_logits
2,2,3499,लाल,زال,laːl,zɒl,Red,still,0.750000,0.333333,...,0.069444,3,hard_negative,0,0.591443,0.788639,"[-1, 1, 1, 1, -1, 1, -1, 0, 1, -1, -1, 1, 1, -...","[-1, -1, 1, 1, -1, -1, -1, 0, 1, -1, -1, 1, 1,...",0.629748,0.417709
26,26,50,अज़ीम,آزرم,aziːm,ɒzrm,Azeem,Azram,0.600000,0.250000,...,0.098958,5,hard_negative,0,0.591194,0.668628,"[1, 1, -1, 1, -1, -1, -1, 0, 1, -1, -1, 0, -1,...","[1, 1, -1, 1, -1, -1, -1, 0, 1, -1, -1, 0, -1,...",0.613509,0.472721
29,29,816,ख़ुद,بخرد,xud,bxrd,myself,Buy,0.500000,0.500000,...,0.338542,4,hard_negative,0,0.461543,0.491940,"[-1, -1, 1, 1, -1, -1, -1, 0, -1, -1, -1, -1, ...","[-1, -1, 1, -1, -1, -1, -1, 0, 1, -1, -1, 1, -...",0.559385,0.490335
61,61,1480,जुर्माना,جراتمند,d͡ʒurmaːnaː,d͡ʒrɒtmnd,Fine,Bold,0.545455,0.571429,...,0.226190,8,hard_negative,0,0.609012,0.690388,"[-1, -1, 1, -1, 1, -1, -1, 0, 1, -1, -1, -1, 1...","[-1, -1, 1, -1, 1, -1, -1, 0, 1, -1, -1, -1, 1...",0.563029,0.302091
88,88,1326,ज़हमत,زعامت,zəɦmət,zʔɒmt,trouble,زعاط,0.500000,0.500000,...,0.250000,5,hard_negative,0,0.511197,0.666446,"[-1, -1, 1, 1, -1, -1, -1, 0, 1, -1, -1, 1, 1,...","[-1, -1, 1, 1, -1, -1, -1, 0, 1, -1, -1, 1, 1,...",0.606891,0.454403
123,123,3953,साज़िश,داداش,saːziʃ,dɒdɒʃ,intrigue,Brother,0.833333,0.000000,...,0.083333,6,hard_negative,0,0.511790,0.677654,"[-1, -1, 1, 1, -1, -1, -1, 0, -1, -1, -1, 1, 1...","[-1, -1, 1, -1, -1, -1, -1, 0, 1, -1, -1, 1, 1...",0.628664,0.492967
139,139,1749,तोती,تبتل,totiː,tbtl,parrot,celibacy,0.600000,0.500000,...,0.161458,4,hard_negative,0,0.587399,0.467504,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ...","[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ...",0.602100,0.401709
145,145,250,ख़ान,خاتون,xaːn,xɒtvn,Khan,Woman,0.600000,0.400000,...,0.425000,5,random,0,0.550290,0.835323,"[-1, -1, 1, 1, -1, -1, -1, 0, -1, -1, -1, -1, ...","[-1, -1, 1, 1, -1, -1, -1, 0, -1, -1, -1, -1, ...",0.622566,0.505698
153,153,2371,फ़रमान,غیرممکن,fərmaːn,ɣjrmmkn,decree,impossible,0.571429,0.571429,...,0.247024,7,hard_negative,0,0.414517,0.554409,"[-1, -1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -...","[-1, -1, 1, 1, -1, -1, -1, 0, 1, -1, -1, -1, -...",0.559431,0.374129
160,160,3411,रूम,عام,ruːm,ʔɒm,room,general,0.750000,0.000000,...,0.173611,3,hard_negative,0,0.319485,0.478434,"[-1, 1, 1, 1, 0, -1, -1, 0, 1, -1, -1, 1, 1, -...","[-1, 1, -1, -1, -1, -1, -1, 0, -1, -1, 1, -1, ...",0.610450,0.478433


In [346]:
test.iloc[fn,:]

,Unnamed: 0,Unnamed: 0.1,loan_word,original_word,loan_word_epitran,original_word_epitran,loan_english,original_english,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,...,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,label,label_bin,mbert_cos_similarity,xlm_cos_similarity,features_loan,features_orig,DNN_logits,CNN_logits
84,84,1121,वलीअहद,ولی‌عهد,vəliːaɦəd,vlj‌ʔhd,वलीअहद,Prince,0.666667,0.500000,...,0.302083,7,loan,1,0.305544,0.743281,"[-1, -1, 1, 1, -1, -1, -1, 1, 1, -1, -1, 1, -1...","[-1, -1, 1, 1, -1, -1, -1, 1, 1, -1, -1, 1, -1...",0.916168,0.925267
128,128,184,क़ैंची,قیچی,qæːɲt͡ʃiː,ɣjt͡ʃj,scissors,scissor,0.666667,0.600000,...,0.275000,6,loan,1,0.503765,0.687581,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, -1,...","[-1, -1, 1, 1, -1, -1, -1, 0, 1, -1, -1, -1, -...",0.898534,0.834707
189,189,893,मक्का,مکه,məkkaː,mkh,Maize,مکه,0.666667,0.600000,...,0.433333,5,loan,1,0.547034,0.556223,"[-1, 1, 1, -1, -1, -1, 1, 0, 1, -1, -1, 1, -1,...","[-1, 1, 1, -1, -1, -1, 1, 0, 1, -1, -1, 1, -1,...",0.890917,0.808590
219,219,134,ऐनक,عینک,æːnək,ʔjnk,Glasses,Glasses,0.600000,0.750000,...,0.218750,4,loan,1,0.616185,0.585316,"[1, 1, -1, 1, -1, -1, -1, 0, 1, -1, -1, 0, -1,...","[-1, 1, -1, -1, -1, -1, -1, 0, -1, -1, 1, -1, ...",0.902081,0.811025
222,222,180,क़ीमती,قیمتی,qiːmtiː,ɣjmtj,treasured,قیمتی,0.714286,0.400000,...,0.075000,6,loan,1,0.305205,0.614641,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, -1,...","[-1, -1, 1, 1, -1, -1, -1, 0, 1, -1, -1, -1, -...",0.904619,0.817428
234,234,665,नशा,نشئه,nəʃaː,nʃjʔh,Intoxication,نشئه,0.800000,0.800000,...,0.329167,4,loan,1,0.600376,0.645460,"[-1, 1, 1, -1, -1, -1, 1, 0, 1, -1, -1, 1, 1, ...","[-1, 1, 1, -1, -1, -1, 1, 0, 1, -1, -1, 1, 1, ...",0.903961,0.831530
266,266,182,क़ुव्वत,قوت,quvvət,ɣvt,Quvvat,قوت,0.666667,0.500000,...,0.520833,7,loan,1,0.360820,0.540769,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, -1,...","[-1, -1, 1, 1, -1, -1, -1, 0, 1, -1, -1, -1, -...",0.897682,0.897601
337,337,1299,हम-,هم,ɦəmə-,hm,we-,also,0.800000,0.500000,...,0.510417,3,loan,1,0.451292,0.499357,"[-1, 1, 1, 1, -1, -1, -1, 0, 1, -1, -1, -1, -1...","[-1, 1, 1, 1, -1, -1, -1, 0, -1, -1, -1, -1, -...",0.899792,0.855186
358,358,687,निगरानी,نگرانی,niɡraːniː,ŋrɒnj,Surveillance,Worry,0.777778,0.428571,...,0.333333,7,loan,1,0.602658,0.823523,"[-1, 1, 1, -1, -1, -1, 1, 0, 1, -1, -1, 1, 1, ...","[-1, 1, 1, -1, -1, -1, 1, 0, 1, -1, -1, -1, -1...",0.898913,0.833034
382,382,36,अर्क़,عرق,arqə,ʔrɣ,extract,Sweat,0.750000,0.500000,...,0.343750,5,loan,1,0.523397,0.637884,"[1, 1, -1, 1, -1, -1, -1, 0, 1, -1, -1, 0, -1,...","[-1, 1, -1, -1, -1, -1, -1, 0, -1, -1, 1, -1, ...",0.886994,0.790175


# SVM

In [347]:
from sklearn import svm

In [348]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(x_train, y_train)

SVC(kernel='linear')

In [349]:
y_pred = svclassifier.predict(x_test)

In [350]:
print("f1-score : ", f1_score(y_test, y_pred ))
print("precision : ",precision_score(y_test, y_pred))
print("recall : ",recall_score(y_test, y_pred )) 
print("accuracy : ",accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

f1-score :  0.9158249158249158
precision :  0.8831168831168831
recall :  0.951048951048951
accuracy :  0.9458874458874459
              precision    recall  f1-score   support

           0       0.98      0.94      0.96       319
           1       0.88      0.95      0.92       143

    accuracy                           0.95       462
   macro avg       0.93      0.95      0.94       462
weighted avg       0.95      0.95      0.95       462



# random forest


In [351]:
from sklearn.ensemble import RandomForestClassifier

In [352]:
RF = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)

In [353]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((4200, 9), (4200,), (462, 9), (462,))

In [354]:
RF.fit(x_train, y_train)

RandomForestClassifier(max_depth=5, random_state=0)

In [355]:
y_pred = RF.predict(x_test)

In [356]:
print("f1-score : ", f1_score(y_test, y_pred ))
print("precision : ",precision_score(y_test, y_pred))
print("recall : ",recall_score(y_test, y_pred )) 
print("accuracy : ",accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

f1-score :  0.9215017064846417
precision :  0.9
recall :  0.9440559440559441
accuracy :  0.9502164502164502
              precision    recall  f1-score   support

           0       0.97      0.95      0.96       319
           1       0.90      0.94      0.92       143

    accuracy                           0.95       462
   macro avg       0.94      0.95      0.94       462
weighted avg       0.95      0.95      0.95       462

[[304  15]
 [  8 135]]
